# Python 导入其他文件夹中的 Module 规则

在 Python 中导入其他文件夹中的模块（module）时，有几个规则和方法可供使用。主要涉及 Python 的 **模块搜索路径（sys.path）**，以及如何使用 **相对导入** 和 **绝对导入**。

## **1. Python 如何查找模块**
当 `import module_name` 运行时，Python 按照以下顺序查找模块：
1. **当前目录**（脚本所在的文件夹）。
2. **`sys.path` 中的目录**：
   - `PYTHONPATH` 环境变量中的路径。
   - Python 安装目录中的标准库路径。
   - `site-packages` 目录（安装的第三方库）。

你可以运行以下代码查看 Python 查找模块的路径：


## **2. 导入其他文件夹中的模块的方法**
如果你的模块不在默认的搜索路径中，你可以使用以下方法导入：

### **方法 1：使用 `sys.path.append()`（动态导入）**
如果你想导入 `other_folder/module.py`，可以在代码中手动添加路径：


In [1]:
import sys
import os

sys.path.append(os.path.abspath("other_folder"))  # 绝对路径
import module  # 现在可以正常导入


ModuleNotFoundError: No module named 'module'

### **方法 2：使用 `PYTHONPATH` 环境变量**
如果你不想在代码中修改 `sys.path`，可以在环境变量中设置 `PYTHONPATH`：

```bash
export PYTHONPATH=$PYTHONPATH:/path/to/other_folder
```

然后你可以直接 `import module`。

### **方法 3：使用 `package` 和 `__init__.py`（推荐方式）**
另外一个比较好的做法是将想要导入的文件夹变成 **包（package）** 结构：

#### **目录结构**
```
project_root/
│── main.py
│── my_package/
│   │── __init__.py
│   │── my_module.py
```
#### **导入方式**
在 `main.py` 中：


```{code} python3
from my_package import my_module
```

### **方法 4：使用 `importlib` 动态导入**
如果你只知道模块路径，但不想改动 `sys.path`，可以使用 `importlib`：


In [ ]:
import importlib.util

module_path = "other_folder/module.py"
module_name = "module"

spec = importlib.util.spec_from_file_location(module_name, module_path)
module = importlib.util.module_from_spec(spec)
spec.loader.exec_module(module)

# 现在可以使用 module 了
module.some_function()


### **方法 5：使用 `setuptools` 安装本地模块**
如果你的模块在 `my_package/` 目录下，并且你希望以 Python 包的形式导入，可以在 `setup.py` 中定义：

```python
from setuptools import setup, find_packages

setup(
    name="my_package",
    version="0.1",
    packages=find_packages(),
)
```

然后运行：
```bash
pip install -e .
```

这样，`my_package` 就可以像普通 Python 库一样被导入：


In [ ]:
import my_package.my_module


## **3. 相对导入 vs 绝对导入**
如果你的模块在 **同一个 package** 内，使用 **相对导入** 会更方便：

```python
# 在 my_package/my_module.py 中
from . import another_module  # 相对导入
```

但在 `main.py` 运行时，**相对导入可能会失败**，此时需要使用绝对导入：

```python
from my_package import my_module
```

### **相对导入适用于：**
- 同一个包（package）内的模块导入。
- 需要更便捷地组织代码，不依赖全局 `sys.path`。

---

## **4. 总结**
| 方法 | 适用场景 | 代码示例 |
|------|------|------|
| **修改 `sys.path`** | 临时添加路径 | `sys.path.append("/path/to/module")` |
| **环境变量 `PYTHONPATH`** | 适用于开发环境 | `export PYTHONPATH=$PYTHONPATH:/path/to/module` |
| **Python package 结构（推荐）** | 组织良好的项目 | `from my_package import my_module` |
| **`importlib` 动态加载** | 需要动态导入模块 | 使用 `importlib.util` |
| **`setuptools` 方式** | 需要安装本地模块 | `pip install -e .` |

如果是 **正式项目**，推荐使用 **方法 3（package 结构）** 或 **方法 5（setuptools 安装）**，避免修改 `sys.path`。

如果你的 `import` 失败，可以先 `print(sys.path)` 看看 Python 是否找得到你的包！🚀
